# Final Project - Trump/Hillary Tweets
### Question: 

What combination of words result in the highest amount of retweets? As in, are there a set of #N words that Trump or Hillary could tweet that garners the most retweets and likes?

We are using Trump and Hillary Tweets before the 2016 Presidential Election and analyzing these two datasets to find the most common words that each respective candidates used.

In [37]:
#Basic packages to be used in the project. 
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import string

#Natural Language Toolkit
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords

#NLTK tokenizer for tweets.
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

[nltk_data] Downloading package punkt to /home/trandonbay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/trandonbay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Loading and Cleaning the Data

We are loading both the datasets so we can retrieve both Trump and Hillary tweets.

In [38]:
#Dataframe of Trump tweets.
df_trump = pd.read_csv('Trump_Tweets.csv', encoding='latin-1');

#Dataframe of Hillary tweets.
df_th = pd.read_csv('Trump_Hillary_Tweets.csv');
df_hillary = df_th[df_th['handle'] == 'HillaryClinton'];

In [39]:
#Cleaned unnecessary columns of the Trump tweets.
del df_trump['Unnamed: 10'];
del df_trump['Unnamed: 11'];

In [40]:
#TESTING
#df_trump

In [41]:
#TESTING
#tweet = tknzr.tokenize(df_trump['Tweet_Text'][0])
#tweet = words_stop(tweet)
#tweet = words_only(tweet)
#tweet = words_extra(tweet)
#print(tweet);

## Data Manipulation

In [42]:
#Helper method to filter out stopwords.
def words_stop(tweet_list):
    punctuation = list(string.punctuation)
    stop = stopwords.words('english') + punctuation + ['rt','via']
    return [word for word in tweet_list if word not in stop]

#Helper method to filter out hashtags and mentions.
def words_only(tweet_list):
    return [word for word in tweet_list if not word.startswith(('#','@','û','https'))]

#Helper method to filter extra words.
def words_extra(tweet_list):
    extra = ['\x89','...','…','“','”','’','—']
    return [word for word in tweet_list if word not in extra]

### 1. Parsing the Trump Tweets.

We are parsing the Trump tweets, so we can create a frequency distribution of words contained in his tweets.

In [43]:
#METHOD: Parse Trump tweets and create a frequency distribution of words.

#Tokenizes the Trump tweets.
trump_list = []
for trump_tweets in df_trump['Tweet_Text']:
    trump_list.extend(tknzr.tokenize(trump_tweets))

#Filters the tweets.
trump_list = words_stop(trump_list)
trump_list = words_only(trump_list)
trump_list = words_extra(trump_list)

#Create the frequency distribution.
fdist_t = nltk.FreqDist(trump_list)

In [79]:
#print(fdist_t);
#trump_list

<FreqDist with 8573 samples and 63904 outcomes>


In [84]:
fdist_t.most_common(20)

[('trump', 1173),
 ('great', 1045),
 ('thank', 889),
 ('hillary', 543),
 ('america', 487),
 ('people', 443),
 ('new', 410),
 ('poll', 364),
 ('make', 346),
 ('donald', 321),
 ('clinton', 316),
 ('get', 305),
 ('president', 267),
 ('like', 266),
 ('big', 263),
 ('vote', 257),
 ('tonight', 244),
 ('time', 232),
 ('cruz', 228),
 ('one', 227)]

### 2. Parsing the Hillary Tweets.

We are parsing the Hillary tweets, so we can create a frequency distribution of words contained in her tweets.

In [46]:
#Tokenizes the Hillary tweets.
hillary_list = []
for hillary_tweets in df_hillary['text']:
    hillary_list.extend(tknzr.tokenize(hillary_tweets))

#Filters the tweets.
hillary_list = words_stop(hillary_list)
hillary_list = words_only(hillary_list)
hillary_list = words_extra(hillary_list)

#Create the frequency distribution.
fdist_h = nltk.FreqDist(hillary_list)

In [47]:
#print(fdist);
#hillary_list

In [48]:
fdist_h.most_common(20);

### 3. Dictionary of words with favourites and retweets.

Create a dictionary with the words as the key and a tuple of retweets and favorites, unweighted. Then, weigh retweets more heavily by multiplying by the ratio.

#### Finding the ratio between favorites and retweets.

In [56]:
sum_fav_t = df_trump['twt_favourites_IS_THIS_LIKE_QUESTION_MARK'].sum();
sum_rtwt_t = df_trump['Retweets'].sum();

sum_fav_h = df_hillary['favorite_count'].sum();
sum_rtwt_h = df_hillary['retweet_count'].sum();

trump_ratio = sum_fav_t/sum_rtwt_t
hillary_ratio = sum_fav_h/sum_rtwt_h

print('The ratio of trump favorites to retweet is: ', trump_ratio)
print('The ratio of hillary favorites to retweet is: ', hillary_ratio)

The ratio of trump favorites to retweet is:  2.5952112676056336
The ratio of hillary favorites to retweet is:  2.245588632228205


In [61]:
#Trump tweets dictionary.
from collections import namedtuple
Tweets = namedtuple('Tweets', 'favourites retweets')

trump_dict = {}
i = 0

for trump_tweet in df_trump['Tweet_Text']:
    
    tweet = tknzr.tokenize(trump_tweet)
    tweet = words_stop(tweet)
    tweet = words_only(tweet)
    tweet = words_extra(tweet)
    
    for word in tweet:
        trump_dict.setdefault(word, Tweets(0, 0))
        num_fav = df_trump['twt_favourites_IS_THIS_LIKE_QUESTION_MARK'][i]
        num_rtwt = df_trump['Retweets'][i]
        
        fav = trump_dict[word].favourites + num_fav
        rtwt = trump_dict[word].retweets + (num_rtwt*trump_ratio)
        
        trump_dict[word] = trump_dict[word]._replace(favourites = fav, retweets = rtwt)
        
    i = i + 1

In [72]:
trump_dict;

Tweets(favourites=3654710, retweets=3608828.1228168998)

In [63]:
#Hillary tweets dictionary.
from collections import namedtuple
Tweets = namedtuple('Tweets', 'favourites retweets')

hillary_dict = {}
i = 0

for hillary_tweet in df_hillary['text']:
    
    tweet = tknzr.tokenize(hillary_tweet)
    tweet = words_stop(tweet)
    tweet = words_only(tweet)
    tweet = words_extra(tweet)
    
    for word in tweet:
        hillary_dict.setdefault(word, Tweets(0, 0))
        num_fav = df_hillary['favorite_count'].iloc[i]
        num_rtwt = df_hillary['retweet_count'].iloc[i]
        
        fav = hillary_dict[word].favourites + num_fav
        rtwt = hillary_dict[word].retweets + (num_rtwt*hillary_ratio)
        
        hillary_dict[word] = hillary_dict[word]._replace(favourites = fav, retweets = rtwt)
        
    i = i + 1

In [71]:
hillary_dict;

Tweets(favourites=587221, retweets=595561.54350777133)

In [85]:
#Trump and Hillary dictionaries of reactions/frequency of the words.
rxn_dict_t = {}
rxn_dict_h = {}

for k, v in trump_dict.items():
    rxn_t = v.favourites + v.retweets
    freq_t = fdist_t[k]
    rxn_dict_t.setdefault(k, rxn_t/freq_t)
    
    
for k, v in hillary_dict.items():
    rxn_h = v.favourites + v.retweets
    freq_h = fdist_h[k]
    rxn_dict_h.setdefault(k, rxn_h/freq_h)

In [90]:
#((word.retweets * ratio) + word.favorites) / word.freq
from collections import OrderedDict

trump_rxnfreq = OrderedDict(sorted(rxn_dict_t.items(), key=lambda kv: kv[1], reverse=True))
hillary_rxnfreq = OrderedDict(sorted(rxn_dict_h.items(), key=lambda kv: kv[1], reverse=True))

In [127]:
#Datafram of Trump word frequency vs. word reactions.
df_common_t = pd.DataFrame(fdist_t.most_common(10))
df_rxn_t = pd.DataFrame(list(trump_rxnfreq.items())[0:10])

dfFreq_t = pd.concat([df_common_t,df_rxn_t],axis=1)

dfFreq_t.columns = ['Most Frequently Used', 'Occurrences', 'Most Reacted To', 'Weighted Reactions']
dfFreq_t

,Most Frequently Used,Occurrences,Most Reacted To,Weighted Reactions
0,trump,1173,forensic,561665.396901
1,great,1045,forgotten,428974.044178
2,thank,889,taco,304308.441690
3,hillary,543,bowls,304308.441690
4,america,487,grill,304308.441690
5,people,443,9:45,303221.350986
6,new,410,corps,275065.159155
7,poll,364,chemistry,273712.110986
8,make,346,melanias,223319.582160
9,donald,321,incited,213800.665070


In [146]:
#Dataframe of Hillary word frequency vs. word reactions.
df_common_h = pd.DataFrame(fdist_h.most_common(10))
df_rxn_h = pd.DataFrame(list(hillary_rxnfreq.items())[0:10])

dfFreq_h = pd.concat([df_common_h,df_rxn_h],axis=1)

dfFreq_h.columns = ['Most Frequently Used', 'Occurrences', 'Most Reacted To', 'Weighted Reactions']
dfFreq_h

,Most Frequently Used,Occurrences,Most Reacted To,Weighted Reactions
0,trump,774,delete,1.761127e+06
1,hillary,708,silence,2.538532e+05
2,donald,427,poland,2.148997e+05
3,president,279,account,2.064096e+05
4,america,203,housekeeping,2.025102e+05
5,people,195,so-called,2.020907e+05
6,make,189,losers,2.020907e+05
7,one,181,dummies,2.020907e+05
8,us,174,tested,1.698351e+05
9,trump's,149,hi,1.624831e+05


# Data Visualizations

In [151]:
#Make a histogram. (1-2 graphs)

# Analysis